<a href="https://colab.research.google.com/github/andrew11morozovtwo/telegrambot2/blob/main/bot_i_am_ingeneer_09_01_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
from google.colab import drive
import json
import csv
from datetime import datetime

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from openai import OpenAI
import logging
from telegram import File

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Замените TOKEN на ваш токен API, полученный от BotFather, например: https://web.telegram.org/k/#@_532_bot
TOKEN = 'TOKEN'

# Замените {PROXY_API_KEY} на ваш API-ключ от API.proxyapi.ru
client = OpenAI(
    api_key="- ______________ -",
    base_url="https://api.proxyapi.ru/openai/v1",
)

DEFAULT_RESPONSE_LIMIT = 9

def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Привет! Я бот, который комментирует Ваши посты с помощью API OpenAI. Версия 06 января 2025 года. По умолчанию до 9 комментариев на пост.")

def help(update, context):
    help_text = """
    Чтобы использовать этого бота, выполните следующие шаги:

    1. Добавьте бота в качестве администратора в ваш телеграмм-канал и в чат обсуждения.
    2. Прокомментируйте или дождитесь комментария на любой пост в канале.
    3. Бот автоматически прокомментирует ваш пост с помощью API OpenAI.
    4. Бот будет комментировать до 9 раз каждый пост (по умолчанию).
    5. В конце комментария бот добавит фразу "Комментарий подготовлен ботом-администратором с использованием искусственного интеллекта".
    6. Чтобы изменить количество ответов бота, используйте команду /setlimit <число>.
    7. Чтобы отключить ответы бота, используйте команду /mute.
    8. Чтобы включить ответы бота, используйте команду /unmute.

    Если у вас возникнут вопросы, напишите /start, чтобы получить информацию о боте.
    """
    context.bot.send_message(chat_id=update.effective_chat.id, text=help_text)

def set_response_limit(update, context):
    try:
        new_limit = int(context.args[0])
        if new_limit < 1:
            context.bot.send_message(chat_id=update.effective_chat.id, text="Количество ответов должно быть положительным числом больше нуля. Пример использования: /setlimit 7")
            return
        context.bot_data['response_limit'] = new_limit
        context.bot.send_message(chat_id=update.effective_chat.id, text=f"Количество ответов бота установлено на {new_limit}.")
    except (IndexError, ValueError):
        context.bot.send_message(chat_id=update.effective_chat.id, text="Пожалуйста, укажите корректное число. Пример использования: /setlimit 7")

def mute(update, context):
    context.bot_data['muted'] = True
    context.bot.send_message(chat_id=update.effective_chat.id, text="Бот теперь молчит и не будет публиковать ответы.")

def unmute(update, context):
    context.bot_data['muted'] = False
    context.bot.send_message(chat_id=update.effective_chat.id, text="Бот снова будет публиковать ответы.")

def comment_post(update, context):
    post_id = update.message.reply_to_message.message_id
    user_message = update.message.reply_to_message.caption or update.message.reply_to_message.text
    user_comment = update.message.caption or update.message.text

    if 'comments_count' not in context.user_data:
        context.user_data['comments_count'] = {}

    if post_id not in context.user_data['comments_count']:
        context.user_data['comments_count'][post_id] = 0

    response_limit = context.bot_data.get('response_limit', DEFAULT_RESPONSE_LIMIT)

    if context.user_data['comments_count'][post_id] < response_limit:
        try:
            chat_history = context.user_data.get('chat_history', {}).get(post_id, [])
            chat_history.append({"role": "user", "content": user_comment})

            # Формирование списка сообщений для API
            messages = [
                {"role": "system", "content": "Вы — бот-администратор канала 'Я-Инженер', который делится новостями науки и техники. Ваша задача — давать краткие, вежливые и полезные комментарии к постам, общаясь с читателями от мужского лица на 'ВЫ'. Учитывайте контекст каждого поста и комментарии пользователей."},
                {"role": "user", "content": f"В телеграмм-канале 'Я-Инженер' опубликован пост: '{user_message}'. Пожалуйста, дайте краткий комментарий (не более 400 символов) и используйте эмодзи для повышения привлекательности ответа!"}
            ] + chat_history

            chat_completion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=messages
            )

            comment_text = chat_completion.choices[0].message.content

            if not context.bot_data.get('muted', False):
                send_message_text = f"{comment_text}\n\nКомментарий подготовлен ботом-администратором с использованием искусственного интеллекта."
                context.bot.send_message(chat_id=update.effective_chat.id, text=send_message_text, reply_to_message_id=update.message.reply_to_message.message_id)

            context.user_data['comments_count'][post_id] += 1
            chat_history.append({"role": "assistant", "content": comment_text})
            if 'chat_history' not in context.user_data:
                context.user_data['chat_history'] = {}
            context.user_data['chat_history'][post_id] = chat_history

            # Определяем путь к файлу
            file_path = '/content/gdrive/MyDrive/comments_data_ing.csv'

            # Проверка существования файла и добавление заголовка при необходимости
            if not os.path.exists(file_path):
                with open(file_path, 'w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
                    writer.writerow([
                        "post_id",
                        "date",
                        "user_message",
                        "user_comment",
                        "date_time",
                        "serialized_chat_history",
                        "serialized_messages",
                        "comment_text",
                        "send_message_text"
                    ])

            # Запись данных в файл
            with open(file_path, 'a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)

                # Сериализация chat_history и messages в строку JSON
                serialized_chat_history = json.dumps(chat_history, ensure_ascii=False)
                serialized_messages = json.dumps(messages, ensure_ascii=False)

                # Формирование строки для записи
                current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                data_to_write = [
                    post_id,                                # ID поста
                    datetime.now().strftime("%Y-%m-%d"),    # Дата публикации поста
                    user_message,                           # Текст поста
                    user_comment,                           # Текст комментария
                    current_datetime,                       # Дата и время комментария
                    serialized_chat_history,                # История комментариев
                    serialized_messages,                    # История постов
                    comment_text,                           # Ответ ИИ
                    send_message_text                       # Текст, отправленный ИИ
                ]

                # Запись данных
                writer.writerow(data_to_write)

        except Exception as e:
            logger.error(f"Ошибка при создании комментария: {e}")
            context.bot.send_message(chat_id=update.effective_chat.id, text="Произошла ошибка при создании комментария. Пожалуйста, попробуйте позже.")



        except Exception as e:
            logger.error(f"Ошибка при создании комментария: {e}")
            context.bot.send_message(chat_id=update.effective_chat.id, text="Произошла ошибка при создании комментария. Пожалуйста, попробуйте позже.")


def main():
    updater = Updater(TOKEN, use_context=True)
    dispatcher = updater.dispatcher

    start_handler = CommandHandler('start', start)
    help_handler = CommandHandler('help', help)
    set_limit_handler = CommandHandler('setlimit', set_response_limit, pass_args=True)
    mute_handler = CommandHandler('mute', mute)
    unmute_handler = CommandHandler('unmute', unmute)
    comment_handler = MessageHandler(Filters.reply & (Filters.text | Filters.photo), comment_post)

    dispatcher.add_handler(start_handler)
    dispatcher.add_handler(help_handler)
    dispatcher.add_handler(set_limit_handler)
    dispatcher.add_handler(mute_handler)
    dispatcher.add_handler(unmute_handler)
    dispatcher.add_handler(comment_handler)

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()